In [1]:
#!pip install h2o

In [2]:
import h2o
from h2o.automl import H2OAutoML

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 day 4 hours 14 mins
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_andyg_vutumg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.517 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [5]:
import pandas as pd

In [6]:
stores_train = pd.read_csv('data/stores_train_preprocessed.csv')

In [7]:
stores_train.isnull().sum()

store_id                        0
grunnkrets_id                   0
lat                             0
lon                             0
chain_name                      0
mall_name                       0
revenue                         0
grunnkrets_population         805
district_population            30
distance_from_busstop           0
busstop_importance_level        0
lv1                             0
lv2                             0
lv3                             0
lv4                             0
households_grunnkrets           0
municipality_name               0
area_km2                       30
district_municipality_name      0
dtype: int64

In [8]:
stores_train['grunnkrets_population'] = (stores_train['grunnkrets_population'].fillna((stores_train.grunnkrets_population.mean())))
stores_train['district_population'] = (stores_train['district_population'].fillna((stores_train.district_population.mean())))
stores_train['area_km2'] = (stores_train['area_km2'].fillna(stores_train.area_km2.mean()))

In [9]:
stores_train.isnull().sum()

store_id                      0
grunnkrets_id                 0
lat                           0
lon                           0
chain_name                    0
mall_name                     0
revenue                       0
grunnkrets_population         0
district_population           0
distance_from_busstop         0
busstop_importance_level      0
lv1                           0
lv2                           0
lv3                           0
lv4                           0
households_grunnkrets         0
municipality_name             0
area_km2                      0
district_municipality_name    0
dtype: int64

In [10]:
stores_train = stores_train[stores_train.revenue > 0.0]

In [11]:
import numpy as np
stores_train['revenue'] = (np.log10(stores_train['revenue']))*1000000

In [12]:
data = stores_train

In [14]:
data = data.drop(['store_id','mall_name','busstop_importance_level','lv1','lat','lon','grunnkrets_id','lv2','lv3','lv4'], axis = 1)

In [15]:
data = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
train, test = data.split_frame(ratios=[0.8], seed = 42)

In [17]:
model = H2OAutoML(sort_metric='rmsle',
                  seed = 42,
                  max_models = 20,
                  exclude_algos = ['DeepLearning'],
                 )

In [18]:
model.train(training_frame = train, y = 'revenue')

AutoML progress: |█
22:54:00.446: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_17_20221112_225400


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    49                 49                          45213                  7            7            7             40            103           68.7755

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 235988667257.65466
RMSE: 485786.6478791432
MAE: 364581.7928995966
RMSLE: NaN
Mean Residual Deviance: 235988667257.65466

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 284257025017.9278
RMSE: 533157.5986684685
MAE: 401572.87616323313
RMSLE: NaN
Mean Residual Deviance: 284257025017.9278

Cross-Validation Metrics Summary: 
                        mean         sd           cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  -----------  -----------  ------------  ------------  ------------  ------------  ------------
mae                     401573       6940.76      403142        403571        407647        403923        389580
mean_residual_deviance  2.84258e+11  1.63041e+10  2.74867e+11   2.89433e+11   2.92988e+11   3.02761e+11   2.61242e+11
mse                     2.84258e+11  1.63041e+10  2.74867e+11   2.89433e+11   2.92988e+11   3.02761e+11   2.61242e+11
r2                      0.313923     0.00401728   0.313831      0.309449      0.312586      0.313319      0.320433
residual_deviance       2.84258e+11  1.63041e+10  2.74867e+11   2.89433e+11   2.92988e+11   3.02761e+11   2.61242e+11
rmse                    532981       15374.8      524277        537990        541284        550237        511118
rmsle                   nan          0            nan           nan           nan           nan           nan

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2022-11-12 22:54:05  0.598 sec   0                  643637           499094          4.14269e+11
    2022-11-12 22:54:05  0.626 sec   5                  577112           445807          3.33059e+11
    2022-11-12 22:54:05  0.648 sec   10                 544483           418185          2.96461e+11
    2022-11-12 22:54:05  0.668 sec   15                 528387           404250          2.79193e+11
    2022-11-12 22:54:05  0.688 sec   20                 514768           391767          2.64986e+11
    2022-11-12 22:54:05  0.708 sec   25                 507276           384643          2.57329e+11
    2022-11-12 22:54:05  0.728 sec   30                 501311           378666          2.51313e+11
    2022-11-12 22:54:05  0.748 sec   35                 496335           373965          2.46348e+11
    2022-11-12 22:54:05  0.768 sec   40                 492725           370896          2.42778e+11
    2022-11-12 22:54:05  0.788 sec   45                 488782           367399          2.38908e+11
    2022-11-12 22:54:05  0.808 sec   49                 485787           364582          2.35989e+11

Variable Importances: 
variable                    relative_importance    scaled_importance    percentage
--------------------------  ---------------------  -------------------  ------------
chain_name                  6.178e+15              1                    0.675142
district_population         5.50062e+14            0.0890356            0.0601117
grunnkrets_population       5.45677e+14            0.0883258            0.0596325
households_grunnkrets       4.85401e+14            0.0785694            0.0530455
district_municipality_name  4.42201e+14            0.0715767            

In [19]:
h2o_models = h2o.automl.get_leaderboard(model, extra_columns = "ALL")
h2o_models

model_id,rmsle,rmse,mse,mae,mean_residual_deviance,training_time_ms,predict_time_per_row_ms,algo
GBM_2_AutoML_17_20221112_225400,nan,533158,2.84257e+11,401573,2.84257e+11,210,0.004952,GBM
GBM_5_AutoML_17_20221112_225400,nan,533696,2.84832e+11,402537,2.84832e+11,180,0.004583,GBM
DRF_1_AutoML_17_20221112_225400,nan,563589,3.17632e+11,431203,3.17632e+11,781,0.009393,DRF
GBM_grid_1_AutoML_17_20221112_225400_model_12,nan,533048,2.8414e+11,401794,2.8414e+11,224,0.003286,GBM
GBM_grid_1_AutoML_17_20221112_225400_model_4,nan,550623,3.03186e+11,419120,3.03186e+11,296,0.006844,GBM
GBM_grid_1_AutoML_17_20221112_225400_model_6,nan,539079,2.90606e+11,409737,2.90606e+11,214,0.005176,GBM
GBM_4_AutoML_17_20221112_225400,nan,534867,2.86083e+11,401731,2.86083e+11,202,0.004606,GBM
GBM_grid_1_AutoML_17_20221112_225400_model_7,nan,543285,2.95159e+11,415382,2.95159e+11,380,0.008003,GBM
StackedEnsemble_AllModels_1_AutoML_17_20221112_225400,nan,529653,2.80532e+11,397414,2.80532e+11,661,0.009799,StackedEnsemble
GBM_grid_1_AutoML_17_20221112_225400_model_10,nan,545308,2.97361e+11,414021,2.97361e+11,267,0.005372,GBM


In [20]:
best_model = model.get_best_model()
print(best_model)

Model Details
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_17_20221112_225400


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    49                 49                          45213                  7            7            7             40            103           68.7755

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 235988667257.65466
RMSE: 485786.6478791432
MAE: 364581.7928995966
RMSLE: NaN
Mean Residual Deviance: 235988667257.65466

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 284257025017.9278
RMSE: 533157.5986684685
MAE: 401572.87616323313
RMSLE: NaN
Mean Residual Deviance: 284257025017.9278

Cross-Validation Metrics Su

In [21]:
best_model.model_performance(test)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 301043759435.2039
RMSE: 548674.54782886
MAE: 412247.8694476356
RMSLE: NaN
Mean Residual Deviance: 301043759435.2039

In [22]:
y_pred = model.predict(test)
y_pred

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
222001
1.10375e+06
537122
499775
235303
389442
235303
309071
464540
523461


In [23]:
y_truth = test['revenue']
y_truth

revenue
700704
1.09774e+06
1.15036e+06
728597
-539102
-124360
922829
878234
471732
859379


In [24]:
#y_pred.types

In [25]:
#explain_model = model.explain(frame = test, figsize = (8,6))

In [26]:
from h2o.model.regression import h2o_mean_squared_error

h2o_mean_squared_error(y_truth, y_pred)

301043760069.27435

In [27]:
y_pred.min()

-326181.1694869614

In [28]:
y_truth.min()

-3000000.0

In [29]:
y_pred.max()

1863902.4053650284

In [30]:
y_truth.max()

2341630.358111976

In [31]:
y_pred = 10**(y_pred/1000000)
y_pred 

predict
1.66725
12.6984
3.44447
3.16064
1.71911
2.45156
1.71911
2.03738
2.91434
3.33781


In [32]:
y_truth = 10**(y_truth/1000000)
y_truth

revenue
5.02
12.524
14.137
5.353
0.289
0.751
8.372
7.555
2.963
7.234


In [33]:
y_pred.min()

0.47186615738261506

In [34]:
y_truth.min()

0.001

In [35]:
y_pred.max()

73.09748004034678

In [36]:
y_truth.max()

219.59900000000007

In [37]:
stores_test = pd.read_csv('data/stores_test_preprocessed.csv')

In [38]:
stores_test.isnull().sum()

store_id                        0
grunnkrets_id                   0
lat                             0
lon                             0
chain_name                      0
mall_name                       0
grunnkrets_population         511
district_population            27
distance_from_busstop           0
busstop_importance_level        0
lv1                             0
lv2                             0
lv3                             0
lv4                             0
households_grunnkrets           0
municipality_name               0
area_km2                       27
district_municipality_name      0
dtype: int64

In [39]:
stores_test['grunnkrets_population'] = stores_test['grunnkrets_population'].fillna(stores_test.grunnkrets_population.mean())
stores_test['district_population'] = stores_test['district_population'].fillna(stores_test.district_population.mean())
stores_test['area_km2'] = stores_test['area_km2'].fillna(stores_test.area_km2.mean())

In [40]:
stores_test.isnull().sum()

store_id                      0
grunnkrets_id                 0
lat                           0
lon                           0
chain_name                    0
mall_name                     0
grunnkrets_population         0
district_population           0
distance_from_busstop         0
busstop_importance_level      0
lv1                           0
lv2                           0
lv3                           0
lv4                           0
households_grunnkrets         0
municipality_name             0
area_km2                      0
district_municipality_name    0
dtype: int64

In [41]:
stores_test_id = stores_test.drop(stores_test.iloc[:,1:], axis = 1)
stores_test_id

,store_id
0,914206820-914239427-717245
1,916789157-916823770-824309
2,913341082-977479363-2948
3,889682582-889697172-28720
4,997991699-998006945-417222
...,...
8572,917323003-917383529-844309
8573,917353379-917411824-845904
8574,917072302-917089248-833647
8575,916960557-916993161-829908


In [42]:
stores_test = stores_test.drop(['store_id'], axis = 1)

In [43]:
stores_test.head()

,grunnkrets_id,lat,lon,chain_name,mall_name,grunnkrets_population,district_population,distance_from_busstop,busstop_importance_level,lv1,lv2,lv3,lv4,households_grunnkrets,municipality_name,area_km2,district_municipality_name
0,3012704,59.908672,10.787031,1,0,970.000000,20635.0,5.350000e-08,0,1,11,111,1110,386900,248,0.057027,1316
1,8061401,59.201467,9.588243,20,0,749.582693,540.0,1.030000e-06,1,1,11,111,1110,475200,298,0.165993,343
2,3013917,59.962146,10.924524,20,1,2421.000000,85164.0,1.360000e-05,0,1,11,111,1110,451900,248,0.236628,254
3,16012104,63.420785,10.461091,20,0,149.000000,27247.0,8.340000e-06,0,1,11,111,1110,497400,355,0.983436,1066
4,3014305,59.921102,10.785123,1,0,4921.000000,116474.0,8.450000e-07,0,1,11,111,1110,489800,248,0.449502,375


In [44]:
test_data = h2o.H2OFrame(stores_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [45]:
y_pred_test = model.predict(test_data)
y_pred_test

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


predict
314022
1.078e+06
871411
972086
546865
947455
1.0467e+06
467501
948764
1.00395e+06


In [46]:
y_pred_test_non_log = 10**(y_pred_test/1000000)
y_pred_test_non_log

predict
2.06074
11.9674
7.43723
9.37747
3.52261
8.86044
11.1354
2.93428
8.88718
10.0914


In [47]:
h2o.export_file(y_pred_test_non_log, path = "data/pred03.csv", force = True)

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
